In [1]:
from othello_rules import *
from othello_net import *
from tensorflow.python.framework import ops
from datetime import datetime
from example_states import *
from feature_extractor import *
from training_utils import *
from policy_networks import *
import numpy as np
np.set_printoptions(precision=2)

In [2]:
# Ræsum graphið fyrir tensorflow
ops.reset_default_graph()
graphs        = []
img_datas     = []
train_steps   = []
optimizers    = []
ground_truths = []
losses        = []
pred_ups      = []
keep_probs    = []
score_outs    = []
models        = []
learn_rates   = []
sesses        = []
savers        = []

filters       = (48, 64) #, 80)

#filters = [64]


for filter in filters:
    graph_1, img_data_1, train_step_1, optimizer_1, ground_truths_1, \
                loss_1, pred_up_1, keep_prob_1, learn_rate_1, score_out_1 = policy_net_5_layers(filter)
    graphs.append(graph_1)
    img_datas.append(img_data_1)
    train_steps.append(train_step_1)
    optimizers.append(optimizer_1)
    ground_truths.append(ground_truths_1)
    losses.append(loss_1)
    pred_ups.append(pred_up_1)
    keep_probs.append(keep_prob_1)
    learn_rates.append(learn_rate_1)
    score_outs.append(score_out_1)
    sess_1 = tf.Session(graph=graph_1)
    sesses.append(sess_1)
    savers.append( tf.train.Saver())
    saver_1   = tf.train.Saver()
    sess_1.run(tf.initialize_all_variables())
    model_1 = "models/final_sl/layers_5x" + str(filter) + ".ckpt"
    models.append(model_1)
    if os.path.isfile(model_1):
        saver_1.restore(sess_1, model_1)
    tf.reset_default_graph()
    
    graph_2, img_data_2, train_step_2, optimizer_2, ground_truths_2, \
                loss_2, pred_up_2, keep_prob_2, learn_rate_2, score_out_2 = policy_net_6_layers(filter)
    graphs.append(graph_2)
    img_datas.append(img_data_2)
    train_steps.append(train_step_2)
    optimizers.append(optimizer_2)
    ground_truths.append(ground_truths_2)
    losses.append(loss_2)
    pred_ups.append(pred_up_2)
    keep_probs.append(keep_prob_2)
    learn_rates.append(learn_rate_2)
    score_outs.append(score_out_2)
    sess_2 = tf.Session(graph=graph_2)
    sesses.append(sess_2)
    savers.append( tf.train.Saver())
    saver_2   = tf.train.Saver()
    sess_2.run(tf.initialize_all_variables())
    model_2 = "models/final_sl/layers_6x" + str(filter) + ".ckpt"
    models.append(model_2)
    if os.path.isfile(model_2):
        saver_2.restore(sess_2, model_2)
    tf.reset_default_graph()
    
    graph_3, img_data_3, train_step_3, optimizer_3, ground_truths_3, \
                loss_3, pred_up_3, keep_prob_3, learn_rate_3, score_out_3 = policy_net_7_layers(filter)
    graphs.append(graph_3)
    img_datas.append(img_data_3)
    train_steps.append(train_step_3)
    optimizers.append(optimizer_3)
    ground_truths.append(ground_truths_3)
    losses.append(loss_3)
    pred_ups.append(pred_up_3)
    keep_probs.append(keep_prob_3)
    learn_rates.append(learn_rate_3)
    score_outs.append(score_out_3)
    sess_3 = tf.Session(graph=graph_3)
    sesses.append(sess_3)
    savers.append( tf.train.Saver())
    saver_3   = tf.train.Saver()
    sess_3.run(tf.initialize_all_variables())
    model_3 = "models/final_sl/layers_7x" + str(filter) + ".ckpt"
    models.append(model_3)
    if os.path.isfile(model_3):
        saver_3.restore(sess_3, model_3)
    tf.reset_default_graph()
    
matches = get_all_matches('training/')
lenmatches = len(matches)
print(lenmatches)

# Byrjum þjálfunina

validation_path = "validation/"
#print("starting error:" + str(avg_error(validation_path, sess)))
#print('%s: Step %d: Prediction accuracy = %.2f' % (datetime.now(), 0,
#                                                      prediction_accuracy()/float(60)))
iterations = len(matches)
eta        = 1e-5
prev_stop  = 26000
probs      = 1
print("starting eta: " + str(eta))
for i in range(prev_stop, prev_stop+iterations):
    current_match      = matches[i]
    raw_match_movelist = current_match[8:]
    unpacked_movelist  = unpack('b'*60, raw_match_movelist)
    board              = initialize_game()
    player             = -1
    input_batch        = []
    label_batch        = []    

    
    for move in unpacked_movelist:
        if move == 0:
            break
        input_batch, label_batch = prep_and_append_training_batch(i, label_batch,
                                                                  move, input_batch, board, player)
        board = make_move(board, move, player)
        player = -1 if player is 1 else 1
        legal_moves = find_legal_moves(board, player)
        if len(legal_moves) == 0:
            player = -1 if player is 1 else 1

            
    for m in range(len(models)):
        train_steps[m].run(session=sesses[m], feed_dict={img_datas[m]     : input_batch,
                                                         ground_truths[m] : label_batch,
                                                         keep_probs[m]    : probs,
                                                         learn_rates[m]   : eta})

    #img_data_2, train_step_2, optimizer_2, ground_truths_2, loss_2, pred_up_2, keep_prob_2
    if (i % 100 is 0):
        for m in range(len(models)):
            savers[m].save(sesses[m], models[m])
    if (i % 200 is 0) and (i > 0 and i < 1000+prev_stop) or (i+1) == (iterations+prev_stop) or (i % 500 is 0):  
        print('%s, Step %d' % \
                                     (datetime.now().strftime("%d. %b %H:%M:%S"), i))
        for m in range(len(models)):
            errors, accs = policy_loss(validation_path, sesses[m],
                                         img_datas[m], ground_truths[m], keep_probs[m], losses[m], pred_ups[m])
            print('%s: loss = %.3f, accuracy = %.3f' % \
                                     (models[m],  errors, accs))
            #savers[m].save(sesses[m], models[m])
            

print("done")



117298
starting eta: 1e-05
24. Oct 16:38:54, Step 25500
models/final_sl/layers_5x48.ckpt: loss = 2.215, accuracy = 0.487
models/final_sl/layers_6x48.ckpt: loss = 2.190, accuracy = 0.497
models/final_sl/layers_7x48.ckpt: loss = 2.121, accuracy = 0.492
models/final_sl/layers_5x64.ckpt: loss = 2.184, accuracy = 0.485
models/final_sl/layers_6x64.ckpt: loss = 2.201, accuracy = 0.497
models/final_sl/layers_7x64.ckpt: loss = 2.084, accuracy = 0.522


KeyboardInterrupt: 